In [1]:
import pandas as pd 
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.runnables import chain
from bs4 import BeautifulSoup
import requests
from pydantic import BaseModel, Field

In [2]:
url_dataset = '../../datasets/irrelevantes/dataset_links.csv'

dataset = pd.read_csv(url_dataset)

dataset.head()

,link
0,https://marcosdantas.com/idosa-morre-ao-ser-at...
1,https://reporterpb.com.br/noticia/paraiba/2024...
2,https://thmais.com.br/giro-de-noticias/com-exa...
3,https://oglobo.globo.com/saude/epoca/noticia/2...
4,https://www.cnnbrasil.com.br/economia/negocios...


In [3]:
dataset['link'][0]

'https://marcosdantas.com/idosa-morre-ao-ser-atacada-por-pitbull-que-cuidava-em-sp/'

In [4]:
def fetch_page_content(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        text = soup.get_text(separator="\n").strip()
        return text if text else "Conteúdo vazio ou indisponível."
    except Exception as e:
        return f"Erro ao acessar o link: {e}"

In [5]:
class Response(BaseModel):
    summary: str = Field(description="Resumo da notícia fornecida.", required=True)

In [13]:
summary_prompt = PromptTemplate(
    input_variables=["content"],
    template="""
Você é um assistente especializado em gerar resumos detalhados.

Seu objetivo é fornecer um resumo claro e completo, destacando os principais pontos, argumentos e informações relevantes do conteúdo. O resumo deve ser informativo e bem estruturado, com foco nas ideias principais. 

Lembre-se de que o resumo deve ser coeso e coerente, respeitando um limite máximo de 350 palavras, para permitir um pouco mais de profundidade e detalhes.

Texto:
{content}
"""
)

llm = ChatOllama(model="llama3.1", format="json", temperature=0.1)

structured_llm = llm.with_structured_output(Response)

summary_chain = summary_prompt | structured_llm


In [7]:
def summarize_web_content(url):
    page_content = fetch_page_content(url)
    
    if "Erro" in page_content or "Conteúdo vazio" in page_content:
        return page_content  
    
    summary = summary_chain.invoke(page_content)
    return summary


In [14]:
url = dataset['link'][0]
response = summarize_web_content(url)

In [15]:
response.summary

'Uma idosa de 80 anos morreu em São Paulo após ser atacada por um cachorro da raça Pitbull que ela cuidava. O tutor do animal, de 27 anos, está sendo investigado e disse que o cão estava preso, mas conseguiu escapar e atacou a vítima. A polícia está apurando as circunstâncias do caso e o tutor pode ser responsabilizado criminalmente. O incidente ocorreu no bairro do Grajaú, zona sul da capital paulista.'